In [49]:
# building the dataframe for churn analysis

import pandas as pd
import numpy as np

np.random.seed(42)

#Simulate 500 customers
n = 500
df = pd.DataFrame({
    "Age" : np.random.randint(18, 70, n),
    "ContractMonth" : np.random.randint(1, 48, n),
    "MonthlySpend" : np.random.randint(30, 300, n),
    "SupportCalls" : np.random.randint(0, 10, n),
    "PremiumPlan" : np.random.choice([0,1], size=n)
})

#Target Variable: churn
df["Churn"] = (
    (df["ContractMonth"] < 6) |
    (df["SupportCalls"] < 5) |
    (df["PremiumPlan"] == 0) & (df["MonthlySpend"] > 200)
).astype(int)

df.describe()

,Age,ContractMonth,MonthlySpend,SupportCalls,PremiumPlan,Churn
count,500.000000,500.000000,500.00000,500.000000,500.000000,500.000000
mean,44.220000,23.970000,162.77400,4.468000,0.516000,0.620000
std,15.036082,13.381451,78.68434,2.905488,0.500244,0.485873
min,18.000000,1.000000,30.00000,0.000000,0.000000,0.000000
25%,32.000000,12.000000,92.00000,2.000000,0.000000,0.000000
50%,45.000000,24.500000,163.00000,4.000000,1.000000,1.000000
75%,57.000000,35.000000,229.00000,7.000000,1.000000,1.000000
max,69.000000,47.000000,299.00000,9.000000,1.000000,1.000000


In [51]:
import plotly.express as px

fig = px.histogram(df, x="PremiumPlan", color="Churn", template="plotly_dark")
fig.show()

In [14]:
fig = px.histogram(df, x="MonthlySpend", color="Churn", template="plotly_dark")
fig.show()

In [50]:
fig = px.histogram(df, x="ContractMonth", color="Churn", template="plotly_dark")
fig.show()

In [48]:
fig = px.histogram(df, x="SupportCalls", color="Churn", template="plotly_dark")
fig.show()

In [54]:
from sklearn import linear_model
from sklearn import model_selection
from sklearn import metrics

reg = linear_model.LogisticRegression(max_iter=1000)

X = df[["Age","ContractMonth","MonthlySpend","SupportCalls","PremiumPlan"]]
y = df["Churn"]

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3, random_state=42)

reg.fit(X_train, y_train)

y_pred = reg.predict(X_test)
y_prob = reg.predict_proba(X_test)[:,1]

print("Accuracy: \n", metrics.accuracy_score(y_test,y_pred))
print("Confusion Matrix: \n", metrics.confusion_matrix(y_test,y_pred))
print("Classification Report: \n", metrics.classification_report(y_test,y_pred))


Accuracy: 
 0.8333333333333334
Confusion Matrix: 
 [[42 18]
 [ 7 83]]
Classification Report: 
               precision    recall  f1-score   support

           0       0.86      0.70      0.77        60
           1       0.82      0.92      0.87        90

    accuracy                           0.83       150
   macro avg       0.84      0.81      0.82       150
weighted avg       0.84      0.83      0.83       150



In [76]:
coefs = pd.DataFrame({
    "Features":X.columns,
    "Coeficients":reg.coef_[0],
    "Odds Ratio": np.exp(reg.coef_[0])
}).sort_values(by="Coeficients", ascending=False)

coefs

,Features,Coeficients,Odds Ratio
2,MonthlySpend,0.007764,1.007794
0,Age,-0.006198,0.993821
1,ContractMonth,-0.074682,0.928039
3,SupportCalls,-0.902460,0.405571
4,PremiumPlan,-1.062089,0.345733
